# **EXPERIMENT 3:** Testing of LLM Inferences to understand which offers optimal results for Agent 1.

# 1. Introduction


*   This experiment is designed to test different free-tier and limited-tier LLMs to evaluate their ability to provide *precise* and *descriptive* answers based on content retrieved by the Fetcher Agent in Haystack.




# Tasks

*   Setting Up the *Fetcher Agent* in Haystack- Based upon previous findings, `DuckDuckGo` seemed to be the most `preferrable search_api` for our objective.

*   Selecting Free & Limited-Tier LLMs- Considered the `gpt-4o-mini` (limited version) and `gemini-1.5 pro`(free tier version) to compare the handoff between results and costing.

*   Analyzing and Comparing Results- Evaluated LLM outputs based on:
    1. **Conciseness** – Is the answer direct and to the point?
    2. **Descriptiveness** – Does it include relevant details?
    3. **Accuracy** – Does it align with the fetched content?
    4. **Relevance** – Does it address the query properly?







# 2. Setup Phase

In [ ]:
pip install haystack-ai


In [ ]:
pip install google-ai-haystack

# Experimentation Phase

1. Gemini-1.5-pro as the LLM

In [ ]:
import os
import google.generativeai as genai
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from duckduckgo_api_haystack import DuckduckgoApiWebSearch

# Load API Key securely
gemini_api_key = os.getenv("GEMINI")
if not gemini_api_key:
    raise ValueError("GEMINI_API_KEY is missing. Please set it as an environment variable.")

# Initialize Gemini API
genai.configure(api_key=gemini_api_key)

class GeminiGenerator:
    def __init__(self, model="gemini-1.5-pro"):
        self.model = model

    def generate(self, prompt):
        model = genai.GenerativeModel(self.model)
        response = model.generate_content(prompt)
        return response.text if response.text else "No response generated."

# Initialize Components
web_search = DuckduckgoApiWebSearch(top_k=10)
link_content = LinkContentFetcher()
html_converter = HTMLToDocument()
prompt_builder = PromptBuilder(template="""
Given the information below:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
Answer question: {{ query }}.
Answer:
""")
gemini = GeminiGenerator(model="gemini-1.5-pro")

# Get user input
company_name = input("Enter the company name: ")
query = (
    f"Analyze {company_name}'s current business, technical, and marketing strategies. "
    f"Provide insights into their key technical domains, ongoing and upcoming projects, "
    f"recently released products, and innovative approaches in marketing. Highlight how these "
    f"strategies have contributed to their growth and how similar strategies could be adapted to scale a business."
)

# Create Haystack pipeline
pipe = Pipeline()
pipe.add_component("search", web_search)
pipe.add_component("fetcher", link_content)
pipe.add_component("converter", html_converter)
pipe.add_component("prompt_builder", prompt_builder)

# Connect components
pipe.connect("search.links", "fetcher.urls")
pipe.connect("fetcher.streams", "converter.sources")
pipe.connect("converter.documents", "prompt_builder.documents")

# Run the pipeline
output = pipe.run(data={"search": {"query": query}, "prompt_builder": {"query": query}})

# Fetch documents
documents = output.get("converter", {}).get("documents", [])

# Prepare prompt for Gemini
doc_texts = "\n".join([doc.content[:1000] for doc in documents])  # Limiting to 1000 chars per doc
final_prompt = f"{doc_texts}\n\n{query}"

# Get response from Gemini
response = gemini.generate(final_prompt)

# Print Results
print("\n" + "-" * 80)
print("Gemini AI Response:")
print(response)
print("-" * 80)

for i, doc in enumerate(documents, start=1):
    print(f"Document {i}:")
    print(f"Title: {doc.meta.get('title', 'N/A')}")
    print(f"Content: {doc.content[:500]}...")  # Print first 500 chars
    print(f"Link: {doc.meta.get('link', 'N/A')}")
    print("-" * 80)



Enter the company name: ZEPTO


/usr/local/lib/python3.11/dist-packages/duckduckgo_api_haystack/duckduckgoapi.py:176: UserWarning: backend='api' is deprecated, using backend='auto'
  results = self.ddgs.text(**payload)



--------------------------------------------------------------------------------
Gemini AI Response:
## ZEPTO: A Deep Dive into Strategies and Growth

Zepto, a quick-commerce grocery delivery startup, has rapidly risen to prominence in India by focusing on speed and convenience. Their success can be attributed to a combination of robust technical infrastructure, targeted marketing, and efficient operations.

**1. Business Strategy:**

* **Hyperlocal Focus:** Zepto targets densely populated urban areas, enabling them to optimize delivery networks and achieve promised delivery times. This micro-market approach allows for efficient resource allocation and minimizes logistical complexities.
* **10-Minute Delivery Promise:** This core value proposition differentiates Zepto from competitors and caters to the growing demand for instant gratification.
* **Data-Driven Approach:** Zepto heavily relies on data analytics to understand consumer behavior, optimize delivery routes, manage inventory,

#  INFERENCES

Conciseness- **7/10**

    Pros:
    1. The response is well-structured and to the point while covering multiple aspects.
    2. Uses bullet points to keep information digestible.

    Cons:

    1. While comprehensive, it may be too detailed for a quick overview.

Descriptiveness- **9/10**

    Pros:
    1. Provides in-depth insights into Zepto’s business, technical, and marketing strategies.

    2. Mentions specific technologies like Python, Java, AWS, and GCP in the tech stack.

    3. Highlights industry-standard practices such as influencer marketing and referral programs.

    Cons:

    1. Some claims, like "Zepto is likely exploring new product categories," are assumptions rather than confirmed facts.

Accuracy- **8/10**

    Pros:
    1. Captures Zepto’s well-known strategies (10-minute delivery, dark stores, hyperlocal focus).
    2. Discusses commonly used logistics and e-commerce strategies in a relevant way.

    Cons:

    1. No direct references or sources are provided, making verification difficult.

Relevance- **8/10**

    Pros:
    1. The response aligns with the general goal of analyzing Zepto’s operations.

    Cons:

    1. Some sections generalize business advice, which might not be relevant to evaluating Zepto specifically.

Hallucination- **6/10**

    Pros:
    1. The response is mostly logical and informed, not purely fabricated.

    Cons:

    1. Some claims appear to be AI-generated inferences rather than sourced facts. Eg.- Tech stack





# Final Verdict: 7.5/10 (Good but needs source validation)
**Strengths:** Well-structured, insightful, and descriptive.

**Weaknesses:** Contains speculative claims, lacks citations, and some generalization.

**Hallucination Risk:** Moderate—many details seem inferred rather than verified.

**To improve, the AI should:**

✔ Provide sources or links where possible.

✔ Clearly separate facts from speculations.

✔ Avoid unnecessary generalizations.

2. chatgpt-4o-mini as the LLM

In [ ]:
import os
from haystack import Pipeline
from haystack.utils import Secret
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.generators import OpenAIGenerator
from duckduckgo_api_haystack import DuckduckgoApiWebSearch

# Initialize Components
web_search = DuckduckgoApiWebSearch(top_k=10)
link_content = LinkContentFetcher()
html_converter = HTMLToDocument()

# Load API Key securely
api_key_secret = Secret.from_env_var("OPENAI_API_KEY")

template = """
Given the information below:
{% for document in documents %}
    {{ document.content }}
{% endfor %}
Answer question: {{ query }}.
Answer:
"""

company_name = input("Enter the company name: ")
query = f"Analyze {company_name}'s current business, technical, and marketing strategies. Provide insights into their key technical domains, ongoing and upcoming projects, recently released products, and innovative approaches in marketing. Highlight how these strategies have contributed to their growth and how similar strategies could be adapted to scale a business in a similar field."

prompt_builder = PromptBuilder(template=template)
llm = OpenAIGenerator(model="gpt-4o-mini", api_key=api_key_secret)

# Create Pipeline
pipe = Pipeline()
pipe.add_component("search", web_search)
pipe.add_component("fetcher", link_content)
pipe.add_component("converter", html_converter)
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)

# Connect components
pipe.connect("search.links", "fetcher.urls")
pipe.connect("fetcher.streams", "converter.sources")
pipe.connect("converter.documents", "prompt_builder.documents")
pipe.connect("prompt_builder.prompt", "llm.prompt")

# Run pipeline
output = pipe.run(data={"search": {"query": query}, "prompt_builder": {"query": query}})
print(output['llm']['replies'][0])


Enter the company name: zepto


/usr/local/lib/python3.11/dist-packages/duckduckgo_api_haystack/duckduckgoapi.py:176: UserWarning: backend='api' is deprecated, using backend='auto'
  results = self.ddgs.text(**payload)


Analyzing Zepto’s current business, technical, and marketing strategies reveals a comprehensive approach that has propelled it to the forefront of the quick commerce industry in India. Here’s a detailed look into various aspects of their strategies and insights into their growth trajectory.

### Business Strategies

1. **Dark Store Model**: 
   - **Description**: Zepto operates dark stores—mini-warehouses not open to the public, strategically placed in urban areas to optimize delivery times. This allows Zepto to fulfill orders quickly, often within 10 minutes, by keeping inventory close to consumers.
   - **Impact**: This model reduces delivery times dramatically and enhances customer satisfaction. It acts as an effective fulfillment center that is uniquely suited to the demands of on-demand grocery delivery.

2. **Targeted Market**: 
   - **Description**: Zepto mainly focuses on urban dwellers, particularly young professionals, students, and families who value speed and convenience.
 

#  INFERENCES

Conciseness- **7.5/10**

    Pros:
    1. The response is detailed and well-structured

    Cons:

    1. Redundancy in descriptions and re-explaining impacts when already clear.

Descriptiveness- **9/10**

    Pros:
    1. Provides enough detail on Zepto’s business, technical, and marketing strategies.

    Suggestion:

    1. While descriptiveness is strong, reducing verbosity in some areas would improve readability.

Accuracy- **8.5/10**

    Pros:
    1. The information appears largely accurate, covering key aspects of Zepto’s strategy.

    Cons:

    1. Some statements about Zepto’s AI and technology use seem generalized and could be backed by more specific data or sources.

Relevance- **8/10**

    Pros:
    1. No off-topic information is included.

    Suggestion:

    1. Prioritize key insights first for better impact.

Hallucination- **7/10**

    Pros:
    1. No major hallucinations were found

    Cons:

    1. some claims about AI, predictive analytics, and gamification seem speculative without clear sources.





# CONCLUSION:
Both the models perform fairly and *do not major advantages over the other.*

However, Gemini can be preferred over GPT in your case.

**Reasons**:-

1. Free & Accessible – No need for paid subscriptions like GPT-4.
2. Google Search Integration – Can fact-check with real-time sources.
3.  Multi-Modal Capabilities – Can analyze PDFs, images, and more if needed.
4. More Control Over Responses – Can be guided with structured prompts.


# SUGGESTIONS FOR GEMINI-1.5 PRO
1. **CONCISENESS-** Use Gemini’s API with a prompt like:
> "Keep the response under [X] words."

2. **DESCRIPTIVENESS**–
> "Make this response more precise and impactful while maintaining clarity."

3. **ACCURACY**– Since Gemini is backed by Google Search, we can prompt:
> "Verify this information using the latest available sources. If uncertain, mention the confidence level."

4. **RELEVANCE** –
> "Ensure every point directly relates to Zepto’s strategy. Remove any unnecessary elaboration."

5. **HALLUCINATION**–
> "Analyze this text and flag any speculative or unverified claims. Suggest corrections where needed."

